In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [4]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [5]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        #  "matbench_expt_gap",
        "matbench_steels",
    ],
)

2023-01-30 16:32:50 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_steels']


In [6]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [7]:
from collections import defaultdict

In [8]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-30 16:32:53 INFO     Dataset matbench_steels already loaded; not reloading dataset.


Upload progress: 100%|██████████| 78.5k/78.5k [00:00<00:00, 68.8Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_163253/train.jsonl: file-szaqcYKoEIry2ZbYbkMWCld9


2023-01-30 16:32:55.996 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675092775,
  "events": [
    {
      "created_at": 1675092775,
      "level": "info",
      "message": "Created fine-tune: ft-mfsF20zSlI9TPivVlFXbxQmy",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-mfsF20zSlI9TPivVlFXbxQmy",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 78515,
      "created_at": 1675092775,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_163253/train.jsonl",
      "id": "file-szaqcYKoEIry2ZbYbkMWCld9",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "s

2023-01-30 16:41:01 INFO     Recorded fold matbench_steels-0 successfully.


...

2023-01-30 17:19:57 INFO     Recorded fold matbench_steels-3 successfully.


Upload progress: 100%|██████████| 78.9k/78.9k [00:00<00:00, 103Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_171957/train.jsonl: file-TTQBDxbAlQsHe2MmbIs39b9T


2023-01-30 17:19:59.295 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675095599,
  "events": [
    {
      "created_at": 1675095599,
      "level": "info",
      "message": "Created fine-tune: ft-ARHeVjic3mWXt5AIXxZJkrMn",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-ARHeVjic3mWXt5AIXxZJkrMn",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 78880,
      "created_at": 1675095598,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230130_171957/train.jsonl",
      "id": "file-TTQBDxbAlQsHe2MmbIs39b9T",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "s

2023-01-30 17:30:05 INFO     Recorded fold matbench_steels-4 successfully.


In [ ]:
mb.to_file("gpt_bench.json.gz")